In [0]:
from keras.applications.xception import Xception

In [40]:
base_model = Xception(weights = 'imagenet', include_top = False, input_shape=(128,128 , 3))

base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [0]:
for layer in base_model.layers[:]:
  layer.trainable = False

In [42]:
from keras.layers import Dense,Dropout,GlobalAveragePooling2D,BatchNormalization
from keras.models import Model

# Fully connected Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = "relu")(x)
x = Dropout(p = 0.4)(x)
prediction = Dense(2,activation = "softmax")(x)

classifier = Model(inputs=base_model.input, outputs=prediction)

classifier.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.4)`
  


In [0]:
# Compiling the model
from keras import optimizers

classifier.compile(loss="categorical_crossentropy", optimizer="adam", metrics = ["accuracy"])

In [44]:
!unzip "dataset1000.zip"

Archive:  dataset1000.zip
replace dataset1000/training_set/dogs/dog.1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [48]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset1000/training_set',
                                                 target_size = (128,128),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('dataset1000/test_set',
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [0]:
# Creating callbacks
from keras.callbacks import ModelCheckpoint,EarlyStopping   

checkpointer = ModelCheckpoint(filepath='cat_dog_transfer_weights2.hdf5', save_best_only=True)

In [50]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 2000,
                         epochs = 5,
                         validation_data = test_set,
                         validation_steps = 2000,
                         callbacks = [checkpointer])

Epoch 1/5
2000/2000 [==============================] - 559s 279ms/step - loss: 0.2684 - acc: 0.8891 - val_loss: 0.3483 - val_acc: 0.9295
Epoch 2/5
2000/2000 [==============================] - 556s 278ms/step - loss: 0.1463 - acc: 0.9435 - val_loss: 0.6746 - val_acc: 0.9186
Epoch 3/5
2000/2000 [==============================] - 684s 342ms/step - loss: 0.0991 - acc: 0.9629 - val_loss: 0.5087 - val_acc: 0.9350
Epoch 4/5
2000/2000 [==============================] - 839s 419ms/step - loss: 0.0822 - acc: 0.9707 - val_loss: 0.6119 - val_acc: 0.9369
Epoch 5/5
2000/2000 [==============================] - 556s 278ms/step - loss: 0.0754 - acc: 0.9738 - val_loss: 0.6691 - val_acc: 0.9345


In [51]:
# load the weights that yielded the best validation accuracy
classifier.load_weights('cat_dog_transfer_weights2.hdf5')

# evaluate test accuracy
score = classifier.evaluate_generator(test_set,steps = 2000)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.2f%%' % accuracy)

Test accuracy: 92.96%


**Thank You**